# Maritime Ports Database Builder

**Ahoy there, adventurous coders and maritime enthusiasts!** <br>⚓️ Set sail on an exciting coding journey with our Maritime Ports Database Builder! <br>🚀 This Python script is your trusty shipmate to explore the vast seas of maritime data. <br>It collects essential shipping information from a maritime website and crafts a robust SQLite database using the power of SQLAlchemy.

## Features:

- 🌊 Scrapes essential shipping data from a maritime website.
- ⚓️ Constructs a robust SQLite database to store countries, ports, and detailed port data.
- 📚 Respects the website's "robots.txt" rules for ethical web scraping.
- 🗺️ Guided navigation through the code with a warm and friendly tone.
- 🌟 Perfect for developers and maritime enthusiasts seeking valuable maritime insights.

Let our database builder be your guiding light to maritime wisdom. 🌍⛵️ Hoist those coding sails and set off on a marvelous journey together! Happy coding and bon voyage! 🚢🌊

Feel free to customize the summary according to your preferences and any additional features you'd like to highlight! 🎉


##### Library

In [10]:
# Modified on 20 April 2022
import requests
from bs4 import BeautifulSoup as bs
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# CREATES THE SQL DATABASE
from sqlalchemy.exc import IntegrityError
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import declarative_base
Base = declarative_base()

In [ ]:
#Database Name : 
Database = 'ShippingDatabase.db'

##### Database Classes

In [ ]:
class Country(Base):
    __tablename__ = 'country'
    countryCode = Column(String, primary_key=True)
    countryName = Column(String)
    numPorts = Column(Float)

class Port(Base):
    __tablename__ = 'port'
    countryCode = Column(String, ForeignKey('country.countryCode'))
    ckey = Column(String, primary_key=True)
    name = Column(String)

class Port_Data(Base):
    __tablename__ = 'port_data'
    id = Column(Integer, primary_key=True)
    ckey = Column(String, ForeignKey('port.ckey'))
    Address = Column(String)
    Port_Authority = Column(String)
    Phone = Column(String)
    Fax = Column(String)
    Email = Column(String)
    Coordinates = Column(String)
    Port_Type = Column(String)
    Port_Size = Column(String)
    Website = Column(String)
    Terminal = Column(String)

##### Functions

In [11]:
def getCountries(URL, session, headers):
    Country_Data = []
    r = session.get(URL, headers=headers)
    if r.status_code == 200 : 
        soup  = bs(r.content, 'html.parser')
        select = soup.find('select', {'id':'country-content'})
        for opt in select.findAll('option')[1:]:
            Country_Data.append({
                'CountryName' : opt.text.strip(), 
                'CountryCode' : opt['value'].strip()
            })
        return Country_Data
def getPorts(countryCode, session, headers):
    shippingMAINURL = 'https://www.searates.com/maritime/ports-map/'

    data = {
        'c': countryCode
    }

    r = session.post(shippingMAINURL, data=data, headers=headers)
    if r.status_code == 200 : 
        data = r.json()
        ports = [ { 'ckey' : x['ckey'], 'name' : x['name']} for x in data['cports']]
        return {
            'numPorts' : len(data['cports']),
            'Ports' : ports
        }
    else:
        return None
def getPortData(PortCode, session, headers):

    # PortCode = 'durres_al'
    PortDataURL = f'https://www.searates.com/port/{PortCode}'

    r = session.get(PortDataURL, headers=headers)

    if r.status_code == 200:
        soup = bs(r.content, 'html.parser')
        dataTab1 = soup.find('div', {'class': 'incoterms-block__wrapper js-adaptive'})
        dataTab2 = dataTab1.find('div', {'class': 'incoterms-block__list'})

        # Find all the paragraphs with class="incoterms-block__item"
        incoterms_items = dataTab2.find_all('p', class_='incoterms-block__item')

        # Create a dictionary to store the extracted information

        info_dict = {}
        PORT_DATA = {}
        # Extract the specific information and store it in the dictionary
        for item in incoterms_items:
            span_elements = item.find_all('span', class_='incoterms-block__text')
            if len(span_elements) == 2:
                key = span_elements[0].get_text().strip()
                # Join all text elements within the span with commas
                value = ', '.join(span_elements[1].stripped_strings)
                # Replace <br/> tags with commas
                value = value.replace('<br/>', ', ')
                info_dict[key] = value

        # Print the extracted information
        for key, value in info_dict.items():
            PORT_DATA[key] = value
        return PORT_DATA
    else:
        return None

##### Main Function Run

In [ ]:
 
engine = create_engine(f'sqlite:///{Database}')
Base.metadata.create_all(engine)
## Database Variables
try:
    engine = engine
except :
    engine = create_engine(f'sqlite:///{Database}')
Session = sessionmaker(bind=engine)
db_session = Session()

## Webscraping Variables
session = requests.Session()

headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
    }

countries = getCountries('https://www.searates.com/maritime', session, headers)

for country in countries:
    
    Name, Code = country['CountryName'], country['CountryCode']
    print('[*] Data for :', Name)
    print('*'*50)
    country_instance = db_session.query(Country).filter_by(countryCode=Code, countryName=Name ).first()
    if not country_instance:
        country_instance = Country(countryCode=Code, countryName=Name, numPorts=0)
        try:
            db_session.add(country_instance)
            db_session.commit() 
        except : 
            engine = create_engine(f'sqlite:///{Database}')
            Session = sessionmaker(bind=engine)
            db_session = Session()
            db_session.add(country_instance)
            db_session.commit() 
            
        print('[*] Added to Database : ', country_instance)
        
    ports = getPorts(Code, session, headers)
    
    time.sleep(1) # Delay to preserve website response time.
    
    for port in ports['Ports']:
        
        portdb = db_session.query(Port).filter_by(countryCode = country_instance.countryCode, ckey=port['ckey'], name = port['name'] ).first()
        
        if not portdb:
            portdb = Port(
                countryCode = country_instance.countryCode,
                ckey = port['ckey'],
                name = port['name']
                )
            db_session.add(portdb)

            country_instance.numPorts = ports['numPorts']

            try:
                db_session.commit()
            except IntegrityError:
                db_session.rollback()
            
            print('[*] Scraping Port : {}'.format(port['name']))
            PortCode = port['ckey']
            portdata = getPortData(PortCode, session, headers)
            # Initialize default values in case a key is missing in portdata
            default_value = 'N/A'

            portdatadb = Port_Data(
                ckey=portdb.ckey,
                Address=portdata.get('Address', default_value),
                Port_Authority=portdata.get('Port Authority', default_value),
                Phone=portdata.get('Phone', default_value),
                Fax=portdata.get('Fax', default_value),
                Email=portdata.get('Email', default_value),
                Coordinates=portdata.get('Coordinates', default_value),
                Port_Type=portdata.get('Port Type', default_value),
                Port_Size=portdata.get('Port Size', default_value),
                Website=portdata.get('Website', default_value),
                Terminal=portdata.get('Terminal', default_value),
            )
            db_session.add(portdatadb)
            db_session.commit()  
        else:
            print(f'[*] This Port : {port} Exists. Skipping...')